In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from catboost import CatBoostClassifier

In [4]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('Test.csv')

In [5]:
X=train.drop('labels',axis=1)

In [6]:
y=train['labels']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.04, random_state=1)

In [19]:
best_params = {
            'bagging_temperature': 0.5,
            'depth': 8,
            'iterations': 1000,
            'l2_leaf_reg': 25,
            'learning_rate': 0.05,
            'sampling_frequency': 'PerTreeLevel',
            'leaf_estimation_method': 'Gradient',
            'random_strength': 0.8,
            'boosting_type': 'Ordered',
            'feature_border_type': 'MaxLogSum',
            'l2_leaf_reg': 50,
            'max_ctr_complexity': 2,
            'fold_len_multiplier': 2
    }

In [20]:
clf=CatBoostClassifier(**best_params,
                               loss_function='Logloss',
                               eval_metric='AUC',
                               nan_mode='Min',
                               thread_count=8,
                               task_type='CPU',
                               verbose=True)

In [39]:
clf.fit(X_train, y_train,verbose_eval=300)

0:	total: 132ms	remaining: 2m 11s
300:	total: 41.3s	remaining: 1m 35s
600:	total: 1m 24s	remaining: 55.8s
900:	total: 2m 5s	remaining: 13.8s
999:	total: 2m 18s	remaining: 0us


In [40]:
y_pred=clf.predict(X_test)

In [41]:
y_pred

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [42]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 0.294884


In [43]:
from sklearn import metrics 
from sklearn.metrics import f1_score

In [44]:
metrics.accuracy_score(y_test,y_pred)*100
#clf.score(X_test,y_test)*100 #91.30434782608695

91.30434782608695

In [50]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1211
           1       0.62      0.51      0.56       146

    accuracy                           0.91      1357
   macro avg       0.78      0.73      0.75      1357
weighted avg       0.91      0.91      0.91      1357



In [51]:
f1_score(y_test,y_pred,average=None)*100

array([95.17973856, 55.63909774])

In [52]:
f1_score(y_test,y_pred,average='macro')*100

75.4094181532262

In [53]:
f1_score(y_test,y_pred,average='micro')*100

91.30434782608695

In [49]:
result=clf.predict(test)
result.shape
result = pd.DataFrame(result, columns=['labels']).to_excel('Catboostclassifier.xlsx')